In [1]:
import json
import os
import re
import urllib

import lxml.html
import requests

In [2]:
BASE_URL = 'https://www.fantasyflightgames.com'

In [3]:
product_response = requests.get(BASE_URL + '/en/products/')

games = json.loads(re.search(r'collection_data = (\[.+\])', product_response.text).groups()[0])

for game in games:
    game_response = requests.get(BASE_URL + game['url'])
    root = lxml.html.fromstring(game_response.text)
    for link in root.xpath('//a[@class="support-item"]'):
        game_name = game['url'].strip('/').split('/')[-1]
        fname = link.attrib['href'].split('/')[-1]
        ddir = os.path.join('data', game_name)
        os.makedirs(ddir, exist_ok=True)
        dfile = os.path.join(ddir, fname)
        if not os.path.isfile(dfile):
            print('Downloading: {} - {}'.format(game_name, fname))
            r = requests.get(link.attrib['href'])
            with open(dfile, 'wb') as o:
                o.write(r.content)

Downloading: a-game-of-thrones-the-board-game-second-edition - agotbg_faq_v2_forweb.pdf


Downloading: a-game-of-thrones-the-card-game-second-edition - agot_lcg_faq_21.pdf


Downloading: a-game-of-thrones-the-card-game-second-edition - gt_melee_format_regulations_v10.pdf


Downloading: arkham-horror-third-edition - arkham_horror_faq.pdf


Downloading: arkham-horror-the-card-game - return_to_the_dunwich_legacy_rules.pdf


...

Downloading: x-wing-second-edition - onlinepoints_genericupgrades_wave3.pdf


Downloading: x-wing-second-edition - onlinepoints_republic.pdf


Downloading: x-wing-second-edition - onlinepoints_separatists.pdf


Downloading: x-wing-second-edition - onlinepoints_firstorder_wave3.pdf


Downloading: x-wing-second-edition - onlinepoints_resistance_january.pdf


In [4]:
archive_response = requests.get(BASE_URL + '/en/more/product-document-archive/')
root = lxml.html.fromstring(archive_response.text)
main_content = root.xpath('//div[@id="minisite-content"]')[0]

In [88]:
for a in main_content.xpath('//a'):
    if a.attrib.has_key('href'):
        dfile = None
        if 'ffg_content' in a.attrib['href'] or '.pdf' in a.attrib['href']:
            game_name = 'unknown'
            if 'ffg_content' in a.attrib['href']:
                game_name = urllib.parse.unquote(a.attrib['href'].split('/ffg_content/')[1].split('/')[0]).replace(' ', '_').lower()
            fname = urllib.parse.unquote(a.attrib['href'].split('/')[-1]).replace(' ', '_')
            ddir = os.path.join('data', 'archive', game_name)
            os.makedirs(ddir, exist_ok=True)
            dfile = os.path.join(ddir, fname)
            if dfile and not os.path.isfile(dfile):
                print('Downloading: {} - {}'.format(game_name, fname))
                r = requests.get(a.attrib['href'])
                with open(dfile, 'wb') as o:
                    o.write(r.content)
        else:
            print(a.attrib['href'])

/en/index/
/en/news/
/en/products/
/en/upcoming/
http://community.fantasyflightgames.com
/en/about/
/en/more/
/en/basket/
/en/more/living-card-games/
/en/more/fantasy-flight-media-center/
/en/more/fantasy-flight-games-professional/
/en/more/print-on-demand/
/en/op/
/en/more/fantasy-flight-games-center/
/en/more/product-document-archive/
/en/more/events/
/en/more/flight-crew/
/en/more/playtesting/
/en/more/freelance-opportunities/
/en/more/
/en/index/
/en/basket/
/en/news/
/en/products/
/en/upcoming/
http://community.fantasyflightgames.com
/en/about/
/en/more/
http://new.fantasyflightgames.com/edge_news.asp?eidn=333
http://store.fantasyflightgames.com/productdetails.cfm?SKU=MC01
http://store.fantasyflightgames.com/productdetails.cfm?SKU=MC01
http://new.fantasyflightgames.com/edge_news.asp?eidn=105
http://new.fantasyflightgames.com/edge_news.asp?eidn=323
http://www.facebook.com/FantasyFlightGames
http://www.twitter.com/ffgames
https://www.instagram.com/fantasyflightgames/
https://www.you